# IAPI Controller Demo

This notebook to demonstrate how we can use pythonnet (http://pythonnet.github.io/) to call Thermo's Instrument Application Programming Interface
 (https://github.com/thermofisherlsms/iapi) libraries from Python and uses a controller to handle the scans.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import glob
import datetime
import time
from pathlib import Path

from IPython import display
import pylab as plt
import numpy as np
import clr

### Test IAPI Mass Spec and Controllers

In [3]:
import sys
sys.path.append('C:\\Users\\joewa\\Work\\git\\vimms')
sys.path.append('C:\\Users\\Vinny\\work\\vimms')

from vimms.MassSpec import IndependentMassSpectrometer, IAPIMassSpectrometer
from vimms.Controller import IdleController, SimpleMs1Controller, TopNController
from vimms.Environment import Environment, IAPIEnvironment
from vimms.Common import *

In [4]:
ref_dir = 'C:\\Users\\joewa\\Work\\git\\vimms\\Interface\\FusionLibrary\\bin\\Debug\\netstandard2.0'

In [5]:
filename = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Data\\multibeers_urine_data\\beers\\fragmentation\\mzML\\Beer_multibeers_1_T10_POS.mzML'
show_console_logs = False

In [6]:
mass_spec = IAPIMassSpectrometer(POSITIVE, ref_dir, filename, show_console_logs)
controller = IdleController()

In [7]:
pbar = False
if pbar:
    set_log_level_warning()
else:
    set_log_level_debug()

In [8]:
env = IAPIEnvironment(mass_spec, controller, 10, progress_bar=pbar, out_dir='results', out_file='iapi_test.mzML')
env.default_scan_params = None
env.run()

INFO   : IAPIEnvironment                : Initialising environment with mass spec <vimms.MassSpec.IAPIMassSpectrometer object at 0x000001E8D319A6A0> and controller <vimms.Controller.IdleController object at 0x000001E8D16FE6D8>
DEBUG  : IAPIMassSpectrometer           : FusionBridge initialising in DEBUG mode. Input mzML is C:\Users\joewa\University of Glasgow\Vinny Davies - CLDS Metabolomics Project\Data\multibeers_urine_data\beers\fragmentation\mzML\Beer_multibeers_1_T10_POS.mzML
DEBUG  : IAPIMassSpectrometer           : Attaching event handlers
INFO   : IdleController                 : Time 0.600548 Received Scan 2 num_peaks=16 rt=0.60 ms_level=2
INFO   : IdleController                 : Time 0.735170 Received Scan 3 num_peaks=20 rt=0.74 ms_level=2
INFO   : IdleController                 : Time 0.869291 Received Scan 4 num_peaks=22 rt=0.87 ms_level=2
INFO   : IdleController                 : Time 1.002920 Received Scan 5 num_peaks=14 rt=1.00 ms_level=2
INFO   : IdleController         